In [1]:
import os
# os.chdir('../')
import pandas as pd
import numpy as np
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC, load_process_MTL
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.utils.tests import check_mlc_results, check_mtr_results, check_dp_results, check_mtl_results
from DeepMTP.main import DeepMTP
from DeepMTP.utils.utils import generate_config
from tqdm import tqdm
from PIL import Image

/home/dimitriosi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Multi-task learning
<img src="../images/mtl_example.png"/>

# load-process-split

In [2]:
# load dataset
data = load_process_MTL('data')
# process and split
train, val, test, data_info = data_process(data, validation_setting='B', verbose=True)
# sanity check
check_mtl_results(train, val, test)

base_dir: data/multi-task_learning-datasets
data_dir: data/multi-task_learning-datasets/dog
Interaction file: triplet format detected
Interaction file: triplet format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: binary

Interaction file: Checking for novel instances... Done
-- Novel instances detected in the test set
Interaction file: Checking for novel targets... Done
-- no Novel targets detected in the test set
Estimating validation setting... Done-- Detected as setting :B

Instance features file: processing features... Done
Instance features file: processing features... Done

-- Same instance ids in the interaction and features files for the train set
-- Same instance ids in the interaction and features files fo

In [3]:
data_info

{'detected_validation_setting': 'B',
 'detected_problem_mode': 'classification',
 'instance_branch_input_dim': None,
 'target_branch_input_dim': 52}

In [4]:
train

{'y': {'data':       instance_id  target_id  value
  0             290          2    1.0
  1             290          3    0.0
  2             290         14    1.0
  3             290         17    1.0
  4             290         21    1.0
  ...           ...        ...    ...
  5441          102         26    0.0
  5442          102         32    1.0
  5443          102         43    1.0
  5444          102         47    1.0
  5445          102         48    1.0
  
  [4916 rows x 3 columns],
  'original_format': 'triplets',
  'instance_id_type': 'int',
  'target_id_type': 'int',
  'missing_values': True},
 'X_instance': {'data':       id                                                dir
  id                                                         
  290  290  data/multi-task_learning-datasets/dog/image/29...
  158  158  data/multi-task_learning-datasets/dog/image/15...
  517  517  data/multi-task_learning-datasets/dog/image/51...
  131  131  data/multi-task_learning-datasets/dog/ima

# Configure and train network

In [5]:
config = generate_config(    
    instance_branch_input_dim = data_info['instance_branch_input_dim'],
    target_branch_input_dim = data_info['target_branch_input_dim'],
    validation_setting = data_info['detected_validation_setting'],
    general_architecture_version = 'dot_product',
    problem_mode = data_info['detected_problem_mode'],
    learning_rate = 0.0001,
    decay = 0,
    batch_norm = False,
    dropout_rate = 0,
    momentum = 0.9,
    weighted_loss = False,
    compute_mode = 'cuda:1',
    train_batchsize = 512,
    val_batchsize = 512,
    num_epochs = 50,
    num_workers = 8,
    metrics = ['aupr', 'auroc'],
    metrics_average = ['macro'],
    patience = 10,

    evaluate_train = True,
    evaluate_val = True,

    verbose = True,
    results_verbose = False,
    use_early_stopping = True,
    use_tensorboard_logger = True,
    wandb_project_name = 'DeepMTP_v2',
    wandb_project_entity = 'diliadis',
    metric_to_optimize_early_stopping = 'loss',
    metric_to_optimize_best_epoch_selection = 'loss',

    instance_branch_architecture = 'CONV',
    use_instance_features = False,
    instance_branch_params = {
        # 'instance_branch_nodes_reducing_factor': 2,
        # 'instance_branch_nodes_per_layer': [123, 100],
        # 'instance_branch_layers': None,
        'instance_branch_conv_architecture': 'resnet',
        'instance_branch_conv_architecture_version': 'resnet101',
        'instance_branch_conv_architecture_dense_layers': 1,
        'instance_branch_conv_architecture_last_layer_trained': 'last',
    },


    target_branch_architecture = 'MLP',
    use_target_features = False,
    target_branch_params = {
        'target_branch_nodes_reducing_factor': 2,
        'target_branch_nodes_per_layer': [512, 256],
        'target_branch_layers': None,
        # 'target_branch_conv_architecture': 'resnet',
        # 'target_branch_conv_architecture_version': 'resnet101',
        # 'target_branch_conv_architecture_dense_layers': 1,
        # 'target_branch_conv_architecture_last_layer_trained': 'last',
    },

    embedding_size = 30,
    comb_mlp_nodes_reducing_factor = 2,
    comb_mlp_nodes_per_layer = [2048, 2048, 2048],
    comb_mlp_layers = None, 

    save_model = True,

    eval_every_n_epochs = 10,
    

    additional_info = {})

In [6]:
config

{'validation_setting': 'B',
 'enable_dot_product_version': True,
 'problem_mode': 'classification',
 'learning_rate': 0.0001,
 'decay': 0,
 'batch_norm': False,
 'dropout_rate': 0,
 'momentum': 0.9,
 'weighted_loss': False,
 'compute_mode': 'cuda:1',
 'num_workers': 8,
 'train_batchsize': 512,
 'val_batchsize': 512,
 'num_epochs': 50,
 'use_early_stopping': True,
 'patience': 10,
 'evaluate_train': True,
 'evaluate_val': True,
 'verbose': True,
 'results_verbose': False,
 'return_results_per_target': False,
 'metric_to_optimize_early_stopping': 'loss',
 'metric_to_optimize_best_epoch_selection': 'loss',
 'instance_branch_architecture': 'CONV',
 'target_branch_architecture': 'MLP',
 'use_instance_features': False,
 'use_target_features': False,
 'use_tensorboard_logger': True,
 'wandb_project_name': 'DeepMTP_v2',
 'wandb_project_entity': 'diliadis',
 'results_path': './results/',
 'experiment_name': None,
 'save_model': True,
 'instance_branch_input_dim': None,
 'target_branch_input_dim

In [7]:
# initialize model
model = DeepMTP(config)
# print(str(model.deepMTP_model))

# train, validate, test
validation_results = model.train(train, val, test)

CURRENT DIR: /data/dimitriosi_datasets/DeepMTP_v2
Selected device: cuda:1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


current_path: b'/data/dimitriosi_datasets/DeepMTP_v2'
current_path: b'/data/dimitriosi_datasets/DeepMTP_v2'
current_path: b'/data/dimitriosi_datasets/DeepMTP_v2'
Starting training...


wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Warning: instance3 has 1 unique true values
Done
Done
Epoch:1... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 0
Epoch:2... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 0
Epoch:3... Done
  Validating... Done
-----------------------------EarlyStopping counter: 3 out of 10---------------------- best epoch currently 0
Epoch:4... Done
  Validating... Done
-----------------------------EarlyStopping counter: 4 out of 10---------------------- best epoch currently 0
Epoch:5... Done
  Validating... Done
-----------------------------EarlyStopping counter: 5 out of 10---------------------- best epoch currently 0
Epoch:6... Done
  Validating... Done
-----------------------------EarlyStopping counter: 6 out of 10---------------------- best epoch currently 0
Epoch:7

test_aupr_macro,▁
test_auroc_macro,▁
train_aupr_macro,▁█
train_auroc_macro,▁█
train_loss,█▄▃▃▃▂▂▁▁▁▁
val_aupr_macro,█▁
val_auroc_macro,█▁
val_loss,▁█▁▃▂▃▃▂▅▃▄
test_aupr_macro,0.70953
test_auroc_macro,0.42363
train_aupr_macro,0.77514


+-------+--------+--------+------------+-------------+
|  mode | #epoch |  loss  | aupr_macro | auroc_macro |
+-------+--------+--------+------------+-------------+
| train |   0    | 0.6494 |   0.731    |    0.4844   |
| train |   1    | 0.5989 |     -      |      -      |
| train |   2    | 0.591  |     -      |      -      |
| train |   3    | 0.5876 |     -      |      -      |
| train |   4    | 0.581  |     -      |      -      |
| train |   5    | 0.5769 |     -      |      -      |
| train |   6    | 0.5726 |     -      |      -      |
| train |   7    | 0.5666 |     -      |      -      |
| train |   8    | 0.5646 |     -      |      -      |
| train |   9    | 0.5625 |     -      |      -      |
+-------+--------+--------+------------+-------------+
+------+--------+--------+------------+-------------+----------------+
| mode | #epoch |  loss  | aupr_macro | auroc_macro | early_stopping |
+------+--------+--------+------------+-------------+----------------+
| val  |   0    |

# Inference

In [ ]:
# generate predictions from the trained model
results, preds = model.predict(train, return_predictions=True ,verbose=True)

# Continue training model

In [ ]:
# this is a minimal configuration needed for HPO methods like Hyperband
config = {
    'verbose': True,
    'num_epochs': 20,
    'num_workers': 8,
    # 'wandb_project_name': None,
    # 'wandb_project_entity': None,
    # 'use_tensorboard_logger': False
}
# initialize the model and load the pretrained weights etc.
new_model = DeepMTP(config, 'add path to model.pt')

new_model.train(train, val, test)